<a href="https://colab.research.google.com/github/MariliaRot/Covid-19/blob/master/COVID_DADOS_DATAFRAME_MS_PAINEL_PREDI%C3%87%C3%95ES_CASOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise exploratoria de dados - COVID19 no Ceará
Objetivo = estudar algoritmos de predição com a base de dados do Ministerio da Saude.

Fonte da base de dados : Painel Corona Virus - Ministerio da Saude (https://covid.saude.gov.br/)

Algoritmos Copiados https://github.com/CSSEGISandData/COVID-19.git

In [ ]:
# Montando o Google Drive para obtenção da base de dados
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#IMPORTANDO BIBLIOTECAS
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
from datetime import datetime, timedelta, date
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy import linalg, optimize

In [ ]:
# IMportando o arquivo CSV do GoogleDrive
caminho_dados_br_originais = '/content/drive/My Drive/Colab Notebooks/PAINEL_MINISTERIO_DA_SAUDE-BR/HIST-COVIDBR-26082020.csv'
caminho_dados_limpos_julho = '/content/drive/My Drive/Colab Notebooks/PAINEL_MINISTERIO_DA_SAUDE-BR/dadosCE-31-07-2020.csv'
caminho_dados_limpos_agosto = '/content/drive/My Drive/Colab Notebooks/PAINEL_MINISTERIO_DA_SAUDE-BR/dadosCE-26-08-2020.csv'

dadosBRbruto = pd.read_csv(caminho_dados_br)
dadosJulho = pd.read_csv(caminho_dados_limpos_julho)
dadosAgosto= pd.read_csv(caminho_dados_limpos_agosto)

In [ ]:
#colocando dados casos acumludados e suas datas em listas
import datetime
xxxdata = dadosJulho['data']
ydata =dadosJulho['casosAcumulado'].to_numpy()
xdata = pd.to_datetime(xxxdata)
mortos = dadosJulho['obitosAcumulado'].to_numpy()

In [ ]:
# Criando função para plotar

import datetime

def plotGraph(X, y, XReal, yReal, algorithm):
    df = pd.DataFrame(
    {'dates': X, 'infected': y.astype(int)})
    df['dates'] = pd.to_datetime(df['dates'])

    fig = plt.figure()
    ax = plt.axes()
    redLine = plt.plot(XReal, yReal, 'r:', label='Casos Confirmados até 31-07-2020)
    df.set_index('dates')['infected'].plot(legend=True, grid=True, label='Predição de Casos')
    plt.title("COVID-19 no Ceará ("+algorithm+")")
    plt.xlabel("Datas")
    plt.ylabel("Numeros de casos")
    
    blue_patch = mpatches.Patch(color='blue', label='Predição de Casos')
    red_patch = mpatches.Patch(color='red', label='Casos Confirmados até 31-07-2020')
    plt.legend(handles=[red_patch, blue_patch])


dates = xdata
infected = ydata



dfForPlot = pd.DataFrame({'date': dates, 'infected': infected})
dfForPlot['date'] = pd.to_datetime(dfForPlot['date'])
dfForPlot.tail(10)



# Definindo periodo para simulação

diasASimular = 30
dfForPlotDaysWithInfected = dfForPlot[dfForPlot['infected'] != 0]
dfForPlotDaysWithInfectedZ= dfForPlot[dfForPlot['deaths'] != 0]

XTrain = np.arange(0, len(dfForPlotDaysWithInfected['date']))
yTrain = dfForPlotDaysWithInfected['infected']

XPredict = np.arange(0, len(XTrain) + diasASimular)

XRealCases = dfForPlotDaysWithInfected['date']
yRealCases = dfForPlotDaysWithInfected['infected']

predictedDays = pd.date_range(dfForPlotDaysWithInfected['date'].iloc[-1] + datetime.timedelta(days=1), periods=diasASimular, freq='D')
X = np.append(dfForPlotDaysWithInfected['date'], predictedDays)

## Regressão linear

In [ ]:
#treino
from sklearn.linear_model import LinearRegression
LRModel = LinearRegression()
LRModel.fit(XTrain.reshape(-1, 1), yTrain)

#predição
y = LRModel.predict(XPredict.reshape(-1, 1)).astype(int)
y = y + y[0] * -1
y

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Regressão Linear")

## Regressão Polinomial grau 2

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 2)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Regressão Polinomial de grau 2")

## Regressão Polinomial grau 3

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 3)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Regressão Polinomial de grau 3")

## Regressão Polinomial grau 4

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 4)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Regressão Polinomial de grau 4")

## Logistic and Exponential Curve

In [ ]:
def logistic_model(x, a, b, c):
    return c / (1 + np.exp(-(x - b) / a))

In [ ]:
def exponential_model(x, a, b, c):
    return a*np.exp(b*(x-c+10))

In [ ]:
###tem que arrumar essa base de dados

import datetime

dfForPlotBrazil = dfForPlot
dfForPlotBrazil = dfForPlotBrazil[dfForPlotBrazil['infected'] > 0]
date_first_infection = dfForPlotBrazil['date'].iloc[0]

dfForPlotBrazil.loc[:,'date'] = dfForPlotBrazil['date'].map(lambda x: (x - date_first_infection).days)
dfForPlotBrazil

In [ ]:
from scipy.optimize import curve_fit
from scipy.optimize import fsolve

x = list(dfForPlotBrazil.iloc[:, 0])
y = list(dfForPlotBrazil.iloc[:, 1])
fit = curve_fit(logistic_model, x, y, maxfev=100000)

sol = int(fsolve(lambda x: logistic_model(x, fit[0][0], fit[0][1], fit[0][2]) - int(fit[0][2]), fit[0][1]))
end = date_first_infection + datetime.timedelta(days=sol)

pred_x = list(range(max(x), sol))
y_l=[logistic_model(i, fit[0][0], fit[0][1], fit[0][2]) for i in x+pred_x]
plt.scatter(list(dfForPlotBrazil.iloc[:, 0]), list(dfForPlotBrazil.iloc[:, 1]), label="Real data", color="red")
plt.plot(x+pred_x, y_l, label="Logistic model")

peakday = date_first_infection + datetime.timedelta(days=fit[0][1])
print("Infection speed     = %.3f" % fit[0][0])
print("Total infected      = %.0f" % fit[0][2])
print("40 dias = ", date_first_infection + datetime.timedelta(days=40))

In [ ]:
x = list(dfForPlotBrazil.iloc[:, 0])
y = list(dfForPlotBrazil.iloc[:, 1])
fit = curve_fit(exponential_model, x, y)

y_e = [exponential_model(i, fit[0][0], fit[0][1], fit[0][2]) for i in x+pred_x]
plt.scatter(list(dfForPlotBrazil.iloc[:, 0]), list(dfForPlotBrazil.iloc[:, 1]), label="Real data", color="red")
plt.plot(x+pred_x, y_e, label="Exponential model")